In [ ]:
!pip install tensorflow_text
!pip install transformers

import os
import re
import time
import string
import numpy as np 
import pandas as pd 
import seaborn as sns
from itertools import groupby
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 27.2 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully u

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("category_df.csv")

In [ ]:
df.shape

(45904, 10)

In [ ]:
df.isnull().sum()

reviewId                    0
content                     0
score                       0
thumbsUpCount               0
reviewCreatedVersion     5093
at                          0
replyContent            43427
repliedAt               43427
predicted_category          0
sentiment                   0
dtype: int64

In [ ]:
df.head()

,content,category
0,woww,USER_EXPERIENCE
1,let me know more details about this,CONTENT
2,i've been using this for a while and there's a...,INTERFACE
3,good,USER_EXPERIENCE
4,mjkobe,USER_EXPERIENCE


In [ ]:
df = pd.get_dummies(df, columns = ['category'], drop_first=False)

Výběr 30 % dat pro zkrácení doby trénování modelu.  
Dříve bylo provedeno mnoho pokusů o trénování modelu na více datech, ale  bezplatná verze GoogleColab není určena pro dlouhé vykonávání kódu, tyto pokusy byly neúspěšné. 

In [ ]:
p = 0.3
short_dataset = df.sample(frac = p, random_state = 42)

# Spliting
p = 0.8
train = short_dataset.sample(frac = p, random_state = 42)
test = short_dataset.drop(train.index)

In [ ]:
print(train.shape)
print(test.shape)

(11017, 5)
(2754, 5)


#### Removing duplicates

In [ ]:
# Odstranění duplicit, pokud se slova v komentáři vyskytují více než 2krát.
def remove_duplicates(text_before):
    my_dict = dict()
    text_after = list()
    for word in text_before.split():
        if word not in my_dict.keys():
            my_dict[word] = 1
        else:
            my_dict[word] = my_dict[word] + 1
    
    for key,value in my_dict.items():
        if value>=2:
            text_after.append(key)
        else:
            text_after.append(key)
    return " ".join(text_after)

#### Délka komentářů

In [ ]:
train['comment_text_clean'] = train['content'].apply(lambda text : remove_duplicates(text))
test['comment_text_clean'] = test['content'].apply(lambda text : remove_duplicates(text))
# délka komentářů
train['comment_text_len'] = train['comment_text_clean'].apply(lambda x : len(x.split())) 

In [ ]:
train['comment_len'] = train['comment_text_clean'].str.len()
fig = px.histogram(train, x = 'comment_len',
                   title='Histogram of len of comments',
                   labels={'Number of samples, len of comments'},
                   opacity = 0.8,
                   log_y=True,
                   color_discrete_sequence = ['blue'])
fig.show()

#### Čištění textů

In [ ]:
def clean_text(text):
    """Make text lowercase, remove text in square brackets, remove links
    and remove stop words containing numbers"""
    text = text.lower()                                            # Converts the text to lowercase using regex 
    text = re.sub(r"\[.*?\]","",text)                              # Replace's the text into 'nothing" if text is present inside squre brackets.
    text = re.sub("https?://\S+|www\.\S+","",text)                 # Removes the links from the comments.
    text = re.sub("<.*?>+","",text)                                # Remove unwanted
    text = re.sub("\n","",text)                                    # Remove next line symbols '\n'
    text = re.sub("\w*\d\w*","",text)                              # Takes only albhabet and digits.
    return text
train['comment_text_clean'] = train['comment_text_clean'].apply(lambda text : clean_text(text))
test['comment_text_clean'] = test['comment_text_clean'].apply(lambda text : clean_text(text))

#### Odstranění stop slov
Stop slova jsou často používaná slova, která nepřidávají do textu žádné další informace. Slova jako "the", "is", "a" nemají žádnou hodnotu a pouze přidávají do dat šum.

In [ ]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text
    
train['comment_text_clean'] = train['comment_text_clean'].apply(remove_stopwords)
test['comment_text_clean'] = test['comment_text_clean'].apply(remove_stopwords)

#### Stemming
Wikipedie:  
"[Stematizace](https://cs.wikipedia.org/wiki/Stematizace) (anglicky stemming) je nalezení kmene slova (nepřesně a úžeji taky kořene). Algoritmus se nazývá stemmer. V praxi se stematizace používá například ve vyhledávačích, kde dovoluje vyhledávat bez ohledu na konkrétní tvar. Podobnou operaci lematizace provádí lemmatizátor – tato operace vrací základní tvar slova (tj. lemma)."

In [ ]:
# Stemming the texts
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text
train['comment_text_clean'] = train['comment_text_clean'].apply(stemm_text)
test['comment_text_clean'] = test['comment_text_clean'].apply(stemm_text)

In [ ]:
list_classes = ['category_CONTENT', 'category_INTERFACE', 'category_SUBSCRIPTION', 'category_USER_EXPERIENCE']
target_data = train[list_classes]
train_data = train['comment_text_clean']

####**Fine-tuning a pre-trained BERT model**

BERT (Bidirectional Encoder Representations from Transformers) provides dense vector representations for natural language by using a deep, pre-trained neural network with the Transformer architecture. It was originally published by

- Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova: ["BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding"](https://arxiv.org/abs/1810.04805), 2018.

This TF Hub model uses the implementation of BERT from the TensorFlow Models repository on [GitHub](https://github.com/tensorflow/models/tree/master/official/legacy/bert).  
BERT-Base, Uncased: 12-layer, 768-hidden, 12-heads, 110M parameters

####První krok:  
Definujeme url pro enkodér BERT a url pro model předzpracování, které se načítají z hubu TensorFlow.  
   

*   Do proměnné "encoder" umístíme *předtrénovaný model BERT*.
*   *Model předzpracování* slouží k přípravě vstupního textu tak, aby mohl být vložen do enkodéru BERT. Zahrnuje tokenizaci, mapování tokenů na jejich ID a přidání speciálních tokenů [CLS] a [SEP] na vstup. Tyto speciální tokeny hrají roli v mechanismu pozornosti, který BERT používá.




In [ ]:
# Define BERT encoder url and processing url blocks from tensorflowhub
encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4" 
preprocessing = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

####Druhý krok: 
Vytvoříme instance KerasLayer pro model předzpracování i enkodér BERT, které byly načteny v prvním kroku. To umožňuje snadné použití modelů v modelové architektuře Keras.

Tento krok je nutný, protože model předzpracování a enkodér BERT, které byly načteny v prvním kroku, nejsou nativními modely Keras. Jedná se o moduly TensorFlow Hub, což jsou předtrénované modely, které lze načíst a použít v TensorFlow. Nemají však formát, který by bylo možné použít přímo v architektuře modelu Keras.

Abychom mohli tyto moduly TensorFlow Hub použít v modelu Keras, musíme vytvořit instance KerasLayer class pro model předzpracování a enkodér BERT. Class KerasLayer je obalová metoda, kterou poskytuje TensorFlow Hub a která umožňuje používat moduly TensorFlow Hub jako modely v architektuře Keras.

In [ ]:
bert_preprocessing_model = hub.KerasLayer(preprocessing) # preprocessing step in bert-base model
bert_model = hub.KerasLayer(encoder) # BERT-base model encoder

####Třetí krok:
Tato část kódu vytváří vstupní vrstvu, vrstvu předzpracování a výstupní vrstvu modelu.

```
# text_input
```
`text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')`  
vytváří vstupní vrstvu modelu s `shape=()`, což znamená, že může přijímat vstupy libovolného tvaru,   
`dtype=tf.string`, což znamená, že může přijímat řetězcové vstupy.

```
# preprocessing
```
`preprocessing = bert_preprocessing_model(text_input)` aplikuje model preprocessingu.

```
# output
```
`output = bert_model(preprocessing)` aplikuje model BERT (`bert_model`) na předzpracovaný vstup (`preprocessing`) a získá zakódovanou reprezentaci vstupního textu.


In [ ]:
text_input = tf.keras.layers.Input(shape= (), dtype = tf.string, name = 'text') 
preprocessing = bert_preprocessing_model(text_input)
output = bert_model(preprocessing)


####Čtvrtý krok:
V tomto kroku vytvořte nad výstupem enkodéru BERT lineární vrstvu, což znamená, že do modelu přidává novou vrstvu po enkodéru BERT. Tato nová vrstva přijme jako vstup zakódovanou reprezentaci vstupního textu vytvořenou enkodérem BERT a před vytvořením konečného výstupu modelu na něm provede lineární transformaci. 

Lineární vrstva se skládá ze dvou částí:


1.   Dropout layer: `layer_1 = tf.keras.layers.Dropout(0.1, name = 'dropout')(output['pooled_output'])` Jedná se o regularizační techniku, která pomáhá zabránit overfittingu tím, že během trénování náhodně vyřadí (nastaví na nulu) určité procento vstupních jednotek (10%). Pomáhá snížit závislost na jednom příznaku, takže se model může naučit robustnější reprezentaci příznaků.
2.   Dense layer: `layer_2 = tf.keras.layers.Dense(len(list_classes), activation = 'sigmoid', name = 'output')(layer_1)` Tato vrstva se sigmoidní aktivační funkcí a počtem jednotek rovným počtu kategorií v cílovém atributu. Sigmoidní aktivační funkce se používá k získání pravděpodobnosti každé kategorie. Tato vrstva přebírá jako vstup výstup Dropout vrstvy. 

`model = tf.keras.Model(inputs = [text_input], outputs = [layer_2])`   
Tento řádek kódu vytvoří model Keras pomocí funkčního rozhraní API. Definuje vstupy a výstupy modelu a určuje vrstvy, které budou použity k transformaci vstupů na výstupy.


In [ ]:
layer_1 = tf.keras.layers.Dropout(0.1, name = 'dropout')(output['pooled_output'])
layer_2 = tf.keras.layers.Dense(len(list_classes), activation = 'sigmoid', name = 'output')(layer_1)

model = tf.keras.Model(inputs = [text_input], outputs = [layer_2])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
# Defining hyperparameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

####Pátý krok: 
Tato část kódu se používá k trénování modelu. Jako vstupní data použije train_data a target_data a váhy modelu se aktualizují na základě ztrátové funkce a optimalizátoru definovaného v předchozím kroku.

`model.fit()` je metoda používaná k trénování modelu.

Prvním parametrem jsou `train_data`, což jsou vstupní data, na kterých bude model trénován.

Druhým parametrem jsou `target_data`, což jsou výstupní data neboli kategorii odpovídající vstupním datům.

Třetím parametrem je `batch_size`, který určuje počet vzorků na jednu aktualizaci gradientu. V tomto případě je nastaven na 32, což znamená, že váhy modelu budou aktualizovány po každých 32 vzorcích.

Čtvrtým parametrem je `epochs`, který určuje, kolikrát model projde celou sadu dat. V tomto případě je nastaven na hodnotu 3, což znamená, že model projde celou sadu dat třikrát.

Posledním parametrem je `validation_split`, což je část dat, která bude použita pro validaci. V tomto případě je nastaven na 0,1, což znamená, že 10 % dat bude použito pro validaci a zbývajících 90 % bude použito pro trénování.

In [ ]:
# Traning process with 3 epochs.
model.fit(train_data, target_data, batch_size = 32, epochs = 3, validation_split = 0.1)

Epoch 1/3
310/310 [==============================] - 4540s 15s/step - loss: 0.4542 - accuracy: 0.5305 - val_loss: 0.4158 - val_accuracy: 0.6162
Epoch 2/3
310/310 [==============================] - 4413s 14s/step - loss: 0.4237 - accuracy: 0.5754 - val_loss: 0.3992 - val_accuracy: 0.6062
Epoch 3/3
310/310 [==============================] - 4489s 14s/step - loss: 0.4101 - accuracy: 0.5969 - val_loss: 0.3820 - val_accuracy: 0.6470


In [ ]:
# save and load trained model.
model.save('сlassification_model_0.3')
trained_model = tf.keras.models.load_model('сlassification_model_0.3')